#### Chat with documents with conversational memory

![memory](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

In [1]:
import os
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [2]:
from langchain.embeddings import GooglePalmEmbeddings
embedding = GooglePalmEmbeddings(google_api_key=GOOGLE_API_KEY)

In [3]:
from langchain.vectorstores import Chroma
persist_directory = "./vector_database/chroma/"
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question, k=3)
len(docs)

3

In [5]:
from langchain.chat_models import ChatGooglePalm
llm = ChatGooglePalm(temperature=0, google_api_key=GOOGLE_API_KEY)
llm.predict("Hello world!")

'Hello world! How can I help you today?'

#### Building prompt

In [6]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA

question = "what is the name of the professor?"

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'The professor in the video is Andrew Ng. He is a globally recognized leader in AI and deep learning. He is a co-founder of Coursera and an adjunct professor at Stanford University. He is also the founder of Landing AI, a company that helps businesses adopt AI.\r\n\r\nAndrew Ng is a graduate of MIT and Stanford University. He received his PhD in computer science from Stanford in 1999. He has published over 100 papers in top academic journals and conferences. He is also the author of the popular book "Machine Learning Yearning".\r\n\r\nAndrew Ng is a passionate advocate for the use of AI to solve real-world problems. He believes that AI has the potential to improve the lives of millions of people around the world. He is a frequent speaker at conferences and events on AI. He is also a regular contributor to the popular press.\r\n\r\nAndrew Ng is a visionary leader in the field of AI. He is committed to using AI to make the world a better place. He is a role model for aspiring AI practiti

#### Building memory

In [7]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#### ConversationalRetrievalChain

In [8]:
from langchain.chains import ConversationalRetrievalChain
retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [9]:
question = "Is probability a class topic?"
result = qa({"question": question})

In [10]:
result.keys()

dict_keys(['question', 'chat_history', 'answer'])

In [11]:
result["answer"]

"Yes, probability is a class topic. In fact, it is one of the most important topics in machine learning. Probability is used to model uncertainty, which is a key concept in machine learning. Without probability, it would be very difficult to build accurate machine learning models.\n\nProbability is also used to calculate the confidence of a machine learning model's predictions. This is important because it allows us to understand how likely a model is to be correct.\n\nOverall, probability is a very important topic in machine learning. It is used to model uncertainty, calculate the confidence of a model's predictions, and build accurate machine learning models."

In [12]:
question = "why are those prerequesites needed?"
result = qa({"question": question})
result['answer']

'I agree with your answer. The prerequisites are needed because they provide a foundation for understanding machine learning. Without a strong understanding of probability, statistics, and linear algebra, it would be very difficult to learn machine learning.\n\nI would like to add that the prerequisites are also needed because they help students develop the critical thinking skills that are essential for machine learning. Machine learning is a complex field that requires students to be able to think critically about data and to be able to identify patterns. The prerequisites help students develop these skills.\n\nFinally, the prerequisites are also needed because they help students develop the mathematical skills that are essential for machine learning. Machine learning is a mathematical field that requires students to be able to understand and apply mathematical concepts. The prerequisites help students develop these skills.\n\nOverall, the prerequisites are needed because they provid

In [13]:
question = "who are the TAs?"
result = qa({"question": question})
result["answer"]

"The TAs for the machine learning class are:\n\n* Paul Baumstarck\n* Catie Chang\n* Tom Do\n* Zico Kolter\n* Daniel Ramage\n\nPaul Baumstarck works in machine learning and computer vision. Catie Chang is actually a neuroscientist who applies machine learning algorithms to try to understand the human brain. Tom Do is another PhD student, works in computational biology and in sort of the basic fundamentals of human learning. Zico Kolter is the head TA — he's head TA two years in a row now — works in machine learning and applies them to a bunch of robots. And Daniel Ramage is — I guess he's not here — Daniel applies learning algorithms to natural language processing."

In [14]:
question = "what are their majors?"
result = qa({"question": question})
result["answer"]

'The majors of the TAs for the machine learning class are as follows:\n\n* Paul Baumstarck: Computer Science\n* Catie Chang: Neuroscience\n* Tom Do: Computational Biology\n* Zico Kolter: Computer Science\n* Daniel Ramage: Computer Science\n\nI hope this answers your question. Let me know if you have any other questions.'

In [15]:
question = "what is the course code?"
result = qa({"question": question})
result["answer"]

'The course code for the machine learning class is CS229. It is a graduate-level course offered at Stanford University. The course covers the theoretical foundations of machine learning, as well as practical techniques for building and evaluating machine learning models. The course is taught by Andrew Ng, a world-renowned expert in machine learning.'

In [16]:
question = "when is the assignment deadline?"
result = qa({"question": question})
result["answer"]

'The assignment deadline for CS229 is 11:59 PM on the day of the assignment. For example, if the assignment is due on Monday, it must be submitted by 11:59 PM on Monday. There is no late policy for assignments, so any assignments submitted after the deadline will receive a 0.\r\n\r\nHere are some tips for submitting your assignments on time:\r\n\r\n* Start working on your assignments early. This will give you plenty of time to complete them and avoid last-minute stress.\r\n* Create a schedule and stick to it. This will help you stay on track and make sure you have enough time to complete all of your assignments.\r\n* Break down large assignments into smaller tasks. This will make them seem less daunting and help you stay organized.\r\n* Take breaks when you need them. Working for long periods of time without a break can lead to fatigue and decreased productivity.\r\n* Ask for help if you need it. There are many resources available to help you succeed in CS229, including your TAs, the c

In [17]:
question = "how many lectures are in the course?"
result = qa({"question": question})
result["answer"]

"There are 10 lectures in the machine learning class CS229. The lectures are held on Tuesdays and Thursdays from 10:30am to 11:45am in 318 Soda Hall. The instructor is Andrew Ng. The TAs are Paul Baumstarck, Catie Chang, Tom Do, Zico Kolter, and Daniel Ramage. The prerequisites for the class are:\n\n* CS106A: Introduction to Computer Science\n* CS106B: Data Structures and Algorithms\n* MATH104: Linear Algebra\n\nThe goals of the class are to:\n\n* Convey some of the instructor's excitement about machine learning to the students.\n* Enable the students to apply state-of-the-art machine learning algorithms to whatever problems they are interested in.\n* Make the students well qualified to start doing research in machine learning.\n\nThe logistics of the class are as follows:\n\n* The class meets for 10 lectures, each of which is 50 minutes long.\n* The lectures are held on Tuesdays and Thursdays from 10:30am to 11:45am in 318 Soda Hall.\n* The instructor is Andrew Ng.\n* The TAs are Paul

In [18]:
question = "what is the name of the professor?"
result = qa({"question": question})
result["answer"]

'Andrew Ng is the professor of CS229, which is a machine learning class at Stanford University. He is also the co-founder of Coursera and an adjunct professor at Stanford University. He is a globally recognized leader in AI and deep learning. He is the founder of Coursera, an online education platform that offers courses from top universities around the world. He is also the co-founder of Landing AI, a company that helps businesses adopt AI. He is a recipient of the ACM Prize in Computing, the highest honor in computer science, and the IEEE Longuet-Higgins Prize, the highest honor in computer vision. He is a Fellow of the Association for the Advancement of Artificial Intelligence (AAAI) and the Association for Computing Machinery (ACM). He is a member of the National Academy of Engineering.'

In [19]:
result["chat_history"]

[HumanMessage(content='Is probability a class topic?'),
 AIMessage(content="Yes, probability is a class topic. In fact, it is one of the most important topics in machine learning. Probability is used to model uncertainty, which is a key concept in machine learning. Without probability, it would be very difficult to build accurate machine learning models.\n\nProbability is also used to calculate the confidence of a machine learning model's predictions. This is important because it allows us to understand how likely a model is to be correct.\n\nOverall, probability is a very important topic in machine learning. It is used to model uncertainty, calculate the confidence of a model's predictions, and build accurate machine learning models."),
 HumanMessage(content='why are those prerequesites needed?'),
 AIMessage(content='I agree with your answer. The prerequisites are needed because they provide a foundation for understanding machine learning. Without a strong understanding of probability